In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 작업 공간 DDAMFN++ 으로 변경
%cd /content/drive/MyDrive/project/KDT_kiosk/code/extracode/DDAMFN/DDAMFN++
%pwd

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/DDAMFN/DDAMFN++


'/content/drive/MyDrive/project/KDT_kiosk/code/extracode/DDAMFN/DDAMFN++'

In [4]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [5]:
### 깃허브 설치

# !git clone https://github.com/SainingZhang/DDAMFN

## 추론

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from networks.DDAM import DDAMNet  # DDAMNet 모델 임포트
from tqdm import tqdm
import wandb

In [ ]:
# wandb 접속
wandb.login(key='ec9a405befa33740a45134d63bb3ba44febe52d4')

wandb.init(project="est_final_project")

wandb: Currently logged in as: happybok (happybok-korea). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# 이미지와 라벨을 Wandb에 로깅하는 함수
def log_image_to_wandb(image_path, predicted_emotion, true_emotion):
    # 이미지를 Wandb Image 포맷으로 변환
    img = Image.open(image_path)

    # Wandb에 이미지와 예측/실제 라벨 로깅
    wandb.log({
        "image": wandb.Image(img, caption=f"Predicted: {predicted_emotion}, True: {true_emotion}")
    })

# 이미지 전처리
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((112, 112)),  # 모델이 예상하는 입력 크기
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원 추가
    return image.to(device)

# 감정 예측
def predict_emotion(image_path, model):
    input_tensor = preprocess_image(image_path)

    # 추론 수행
    with torch.no_grad():
        output, _, _ = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_class = probabilities.argmax(dim=1).item()

    # 감정 라벨 정의
    labels = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Angry']

    # 결과 반환
    return labels[predicted_class], probabilities[0][predicted_class].item()

In [ ]:
# 모델 정의

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DDAMNet 모델 초기화 (사전 학습된 가중치 사용)
model = DDAMNet(num_class=7, num_head=2, pretrained=True)  # pretrained=True로 설정하여 사전 학습된 가중치를 로드

# 사전 학습된 체크포인트 로드
checkpoint_path = "/content/drive/MyDrive/project/KDT_kiosk/code/extracode/DDAMFN/DDAMFN++/checkpoints_ver2.0/affecnet7_epoch19_acc0.671.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# 모델을 평가 모드로 설정
model.eval()
model.to(device)

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/DDAMFN/DDAMFN++/networks/DDAM.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net = torch.load(os.path.join('./p

DDAMNet(
  (features): Sequential(
    (0): Conv_block(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (1): Conv_block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (2): Mix_Depth_Wise(
      (conv): Conv_block(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (prelu): PReLU(num_parameters=128)
      )
      (conv_dw): MDConv(
        (mixed_depthwise_conv): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    

- DDAMNet(num_class=7, num_head=2, pretrained=True) <br>
헤드의 수: num_head=2로 설정하면, 모델이 2개의 헤드를 사용하여 입력 데이터를 처리합니다. 이 헤드들은 각기 다른 방식으로 데이터를 분석하거나 특정 특징을 추출한 뒤, 최종적으로 결합하여 더 풍부한 정보를 제공합니다.<br>
모델에서 여러 개의 헤드를 사용하는 이유는 복잡한 특징을 더 잘 포착하고, 다양한 관점에서 데이터를 분석함으로써 모델의 성능을 향상시키기 위함입니다.<br>

### 추론 테스트

In [ ]:
# 예측할 이미지 경로 설정
# 단일 이미지로 추론
image_path = "/content/drive/MyDrive/project/KDT_kiosk/model/pre_yolov8face/minji.jpg"

# 이미지의 감정 추론
predicted_emotion, confidence = predict_emotion(image_path, model)
print(f"Predicted emotion: {predicted_emotion} (Confidence: {confidence:.2f})")

Predicted emotion: Angry (Confidence: 0.21)


### 모델-추론 평가
- EST cropped face img를 얼마나 잘 맞추는지 확인

In [ ]:
import json
import os
import random
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
label_mapping = {
    "panic": "Surprise",
    "happy": "Happy",
    "sadness": "Sad",
    "anger": "Angry"
}

# 라벨 파일 로드
label_file_path = "/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/test_label/test_labels_v1.json"
with open(label_file_path, 'r') as f:
    labels_data = json.load(f)

### 적게 테스트하고 싶을때 랜덤하게 뽑음
labels_data = random.sample(labels_data, 50)

# 예측 및 실제 라벨 리스트 초기화
predicted_labels = []
true_labels = []

# 모든 이미지에 대해 예측 수행 및 라벨 비교
if wandb.run is None:
    wandb.init(project="est_final_project")

for item in tqdm(labels_data):
    image_path = item["image"]
    true_emotion = item["emotion"]

    # 라벨 매핑 적용
    if true_emotion in label_mapping:
        true_emotion = label_mapping[true_emotion]

    # 이미지에 대한 예측 수행
    predicted_emotion, _ = predict_emotion(image_path, model)

    # 예측된 라벨과 실제 라벨을 리스트에 추가
    predicted_labels.append(predicted_emotion)
    true_labels.append(true_emotion)

    # 각 예측을 wandb에 로깅
    log_image_to_wandb(image_path, predicted_emotion, true_emotion)


100%|██████████| 50/50 [00:29<00:00,  1.71it/s]


In [ ]:
# 감정 라벨
emotion_labels = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Angry']

# 라벨을 인덱스로 변환하는 매핑
label_to_index = {label: idx for idx, label in enumerate(emotion_labels)}

# true_labels와 predicted_labels를 인덱스로 변환
true_labels_index = [label_to_index[label] for label in true_labels]
predicted_labels_index = [label_to_index[label] for label in predicted_labels]

# 지표 계산
accuracy = accuracy_score(true_labels_index, predicted_labels_index)
precision = precision_score(true_labels_index, predicted_labels_index, average='weighted')
recall = recall_score(true_labels_index, predicted_labels_index, average='weighted')
f1 = f1_score(true_labels_index, predicted_labels_index, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"f1_score: {f1:.2f}")

Accuracy: 0.50
Precision: 0.93
Recall: 0.50
f1_score: 0.60


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Wandb에 지표들 로깅
wandb.log({
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1_score": f1
})

# Wandb에 혼동 행렬 로깅
wandb.log({
    "confusion_matrix": wandb.plot.confusion_matrix(
        probs=None,  # 확률 대신 예측된 값 사용
        y_true=true_labels_index,
        preds=predicted_labels_index,
        class_names=emotion_labels
    )
})

In [ ]:
# wandb 종료
wandb.finish()

accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.5
f1_score,0.60278
precision,0.92554
recall,0.5


# 데이터셋 구축

In [4]:
import json
import os
import shutil
from PIL import Image, ImageFile

In [ ]:
# utk 총 이미지 개수 확인
utk_test_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_test'
utk_train_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_train'

# 파일 개수 확인
utk_test_img_cnt = len([name for name in os.listdir(utk_test_img_path) if os.path.isfile(os.path.join(utk_test_img_path, name))])
utk_train_img_cnt = len([name for name in os.listdir(utk_train_img_path) if os.path.isfile(os.path.join(utk_train_img_path, name))])
print(f"파일 개수: {utk_test_img_cnt} {utk_train_img_cnt}")

파일 개수: 3252 19695


In [ ]:
# est 총 이미지 개수 확인
est_test_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/test_img'
est_train_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/img'
est_val_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/val_img'

# 파일 개수 확인
utk_test_img_cnt = len([name for name in os.listdir(est_test_img_path) if os.path.isfile(os.path.join(est_test_img_path, name))])
utk_train_img_cnt = len([name for name in os.listdir(est_train_img_path) if os.path.isfile(os.path.join(est_train_img_path, name))])
utk_val_img_cnt = len([name for name in os.listdir(est_val_img_path) if os.path.isfile(os.path.join(est_val_img_path, name))])
print(f"파일 개수: {utk_test_img_cnt} {utk_train_img_cnt} {utk_val_img_cnt}")

파일 개수: 1200 5994 1200


In [ ]:
# EST 감정 어노테이션 분포 개수 확인
# JSON 파일 경로 설정
est_test_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/test_label/test_labels_v1.json'
est_train_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/label/labels_v1.json'
est_val_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/val_label/val_labels_v1.json'


# JSON 파일 읽기
with open(est_test_label_path, 'r') as file:
    data1 = json.load(file)
with open(est_train_label_path, 'r') as file:
    data2 = json.load(file)
with open(est_val_label_path, 'r') as file:
    data3 = json.load(file)

# emotion 종류와 개수 확인
est_test_label_cnt = {}
est_train_label_cnt = {}
est_val_label_cnt = {}

for item in data1:
    emotion = item['emotion']
    if emotion in est_test_label_cnt:
        est_test_label_cnt[emotion] += 1
    else:
        est_test_label_cnt[emotion] = 1

for item in data2:
    emotion = item['emotion']
    if emotion in est_train_label_cnt:
        est_train_label_cnt[emotion] += 1
    else:
        est_train_label_cnt[emotion] = 1

for item in data3:
    emotion = item['emotion']
    if emotion in est_val_label_cnt:
        est_val_label_cnt[emotion] += 1
    else:
        est_val_label_cnt[emotion] = 1

est_test_label_cnt, est_train_label_cnt, est_val_label_cnt

({'panic': 300, 'happy': 300, 'sadness': 300, 'anger': 300},
 {'sadness': 1500, 'happy': 1494, 'panic': 1500, 'anger': 1500},
 {'sadness': 300, 'anger': 300, 'panic': 300, 'happy': 300})

### EST crop 데이터 최종폴더로 이동

In [ ]:
import json
import os
import shutil

In [ ]:
# train 이동

# 기존 경로 설정
train_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/label/labels_v1.json'
train_img_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/img'

# 새롭게 만들 경로 설정
new_train_img_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img'
new_train_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v1.json'

# 새로운 디렉토리 생성
os.makedirs(new_train_img_dir, exist_ok=True)
os.makedirs(os.path.dirname(new_train_label_path), exist_ok=True)

# 기존 라벨 파일 읽기
with open(train_label_path, 'r') as file:
    train_data = json.load(file)

# 새로운 라벨 데이터 저장
new_train_data = []

for item in train_data:
    if item['emotion'] != 'panic':  # 'panic' 감정 제외
        new_train_data.append(item)
        # 해당 이미지를 새로운 디렉토리로 복사
        img_src = item['image']
        img_filename = os.path.basename(img_src)
        img_dst = os.path.join(new_train_img_dir, img_filename)
        shutil.copy(img_src, img_dst)

# 새로운 라벨 파일로 저장
with open(new_train_label_path, 'w') as file:
    json.dump(new_train_data, file, indent=4)

print(f"새로운 라벨 및 이미지 데이터가 저장되었습니다.\n이미지 경로: {new_train_img_dir}\n라벨 경로: {new_train_label_path}")


새로운 라벨 및 이미지 데이터가 저장되었습니다.
이미지 경로: /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img
라벨 경로: /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v1.json


In [ ]:
# val 이동

# 기존 경로 설정 (val 데이터셋)
val_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/val_label/val_labels_v1.json'
val_img_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/val_img'

# 새로운 경로 설정
new_val_img_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img'
new_val_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v1.json'

# 새로운 디렉토리 생성
os.makedirs(new_val_img_dir, exist_ok=True)
os.makedirs(os.path.dirname(new_val_label_path), exist_ok=True)

# 기존 라벨 파일 읽기
with open(val_label_path, 'r') as file:
    val_data = json.load(file)

# 새로운 라벨 데이터 저장 (panic 제외)
new_val_data = []
for item in val_data:
    if item['emotion'] != 'panic':  # 'panic' 감정 제외
        new_val_data.append(item)
        # 해당 이미지를 새로운 디렉토리로 복사
        img_src = item['image']
        img_filename = os.path.basename(img_src)
        img_dst = os.path.join(new_val_img_dir, img_filename)
        shutil.copy(img_src, img_dst)

# 새로운 라벨 파일로 저장
with open(new_val_label_path, 'w') as file:
    json.dump(new_val_data, file, indent=4)

print(f"새로운 val 라벨 및 이미지 데이터가 저장되었습니다.\n이미지 경로: {new_val_img_dir}\n라벨 경로: {new_val_label_path}")


새로운 val 라벨 및 이미지 데이터가 저장되었습니다.
이미지 경로: /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img
라벨 경로: /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v1.json


In [ ]:
# test 이동

# 기존 경로 설정 (test 데이터셋)
test_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/test_label/test_labels_v1.json'
test_img_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/test_img'

# 새로운 경로 설정
new_test_img_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img'
new_test_label_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v1.json'

# 새로운 디렉토리 생성
os.makedirs(new_test_img_dir, exist_ok=True)
os.makedirs(os.path.dirname(new_test_label_path), exist_ok=True)

# 기존 라벨 파일 읽기
with open(test_label_path, 'r') as file:
    test_data = json.load(file)

# 새로운 라벨 데이터 저장 (panic 제외)
new_test_data = []
for item in test_data:
    if item['emotion'] != 'panic':  # 'panic' 감정 제외
        new_test_data.append(item)
        # 해당 이미지를 새로운 디렉토리로 복사
        img_src = item['image']
        img_filename = os.path.basename(img_src)
        img_dst = os.path.join(new_test_img_dir, img_filename)
        shutil.copy(img_src, img_dst)

# 새로운 라벨 파일로 저장
with open(new_test_label_path, 'w') as file:
    json.dump(new_test_data, file, indent=4)

print(f"새로운 test 라벨 및 이미지 데이터가 저장되었습니다.\n이미지 경로: {new_test_img_dir}\n라벨 경로: {new_test_label_path}")


새로운 test 라벨 및 이미지 데이터가 저장되었습니다.
이미지 경로: /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img
라벨 경로: /content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v1.json


In [ ]:
# 잘 이동했는지 확인

final_est_test_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img'
final_est_train_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img'
final_est_val_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img'

# 파일 개수 확인
final_utk_test_img_cnt = len([name for name in os.listdir(final_est_test_img_path) if os.path.isfile(os.path.join(final_est_test_img_path, name))])
final_utk_train_img_cnt = len([name for name in os.listdir(final_est_train_img_path) if os.path.isfile(os.path.join(final_est_train_img_path, name))])
final_utk_val_img_cnt = len([name for name in os.listdir(final_est_val_img_path) if os.path.isfile(os.path.join(final_est_val_img_path, name))])
print(f"파일 개수: {final_utk_test_img_cnt} {final_utk_train_img_cnt} {final_utk_val_img_cnt}")

파일 개수: 900 4494 900


In [ ]:
import json

# 라벨 파일 경로 설정
label_paths = {
    "test": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v1.json',
    "train": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v1.json',
    "val": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v1.json'
}

# age 변환 함수
def transform_age(age):
    if age <= 19:
        return 0
    elif 20 <= age <= 39:
        return 1
    else:
        return 2

# 각 라벨 파일에 대해 age 변환 적용
for label_type, path in label_paths.items():
    # 라벨 파일 읽기
    with open(path, 'r') as file:
        data = json.load(file)

    # age 값 변환 적용
    for item in data:
        item['age'] = transform_age(item['age'])

    # 수정된 데이터를 동일한 파일에 저장
    with open(path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"{label_type} 라벨 파일의 age 필드가 변환되었습니다.")


test 라벨 파일의 age 필드가 변환되었습니다.
train 라벨 파일의 age 필드가 변환되었습니다.
val 라벨 파일의 age 필드가 변환되었습니다.


### AIhub 데이터 최종폴더로 이동

In [ ]:
# 나이 변경 0 1 2
# JSON 파일이 있는 디렉토리 경로 설정
json_dir_path = '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label'

# age 변환 함수
def transform_age(age):
    if age <= 19:
        return 0
    elif 20 <= age <= 39:
        return 1
    else:
        return 2

# 디렉토리 내 모든 JSON 파일 순회
for json_file in os.listdir(json_dir_path):
    if json_file.endswith('.json'):  # JSON 파일만 처리
        file_path = os.path.join(json_dir_path, json_file)

        # JSON 파일 읽기
        with open(file_path, 'r') as file:
            data = json.load(file)

        # age 값 변환 적용
        for item in data:
            item['age'] = transform_age(item['age'])

        # 수정된 데이터를 동일한 파일에 저장
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

        print(f"{json_file} 파일의 age 필드가 변환되었습니다.")

combined_train_annotations.json 파일의 age 필드가 변환되었습니다.
combined_test_annotations.json 파일의 age 필드가 변환되었습니다.
combined_val_annotations.json 파일의 age 필드가 변환되었습니다.


In [2]:
# 손상된 이미지를 처리할 수 있도록 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

# JSON 파일 경로 설정
json_paths = {
    "test": '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label/combined_test_annotations.json',
    "train": '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label/combined_train_annotations.json',
    "val": '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label/combined_val_annotations.json'
}

# 이미지가 저장된 폴더들
source_img_dirs = [
    '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_img/b_surprise_img',
    '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_img/c_netural_img',
    '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_img/d_disgust_img',
    '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_img/f_fear_img'
]

# 이미지가 저장될 목적지 폴더들
destination_dirs = {
    "test": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img',
    "train": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img',
    "val": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img'
}

# 새로운 디렉토리 생성
for dir_path in destination_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# 이미지를 JPG로 변환하고 복사하는 함수
def convert_and_copy_image(image_name, destination):
    for source_dir in source_img_dirs:
        source_path = os.path.join(source_dir, image_name)
        if os.path.exists(source_path):
            try:
                img = Image.open(source_path)
                # JPG 파일명 설정
                new_image_name = os.path.splitext(image_name)[0] + '.jpg'
                new_image_path = os.path.join(destination, new_image_name)
                img.convert('RGB').save(new_image_path, "JPEG")
                return True
            except Exception as e:
                print(f"이미지 변환 중 오류 발생: {image_name}, 오류: {e}")
    return False

# 각 JSON 파일에 대해 이미지 파일 변환 및 복사
for label_type, json_path in json_paths.items():
    with open(json_path, 'r') as file:
        data = json.load(file)

    dest_dir = destination_dirs[label_type]

    for item in data:
        image_name = item['image']
        success = convert_and_copy_image(image_name, dest_dir)

        if success:
            print(f"이미지 {image_name}가 {label_type} 폴더로 JPG로 변환되어 복사되었습니다.")
        else:
            print(f"이미지 {image_name}를 찾을 수 없거나 처리할 수 없습니다.")

print("이미지 변환 및 복사가 완료되었습니다.")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
이미지 20221207_ID3170_C_04_N00018.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221117_ID1756_C_01_N00004.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20230106_ID4361_C_04_N00087.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221121_ID2108_C_03_N00182.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20230109_ID4198_C_01_N00102.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221111_ID1564_C_01_N00087.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221110_ID1478_C_01_N00092.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221219_ID3902_C_03_N00064.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221207_ID3269_C_01_N00072.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221204_ID3082_C_01_N00022.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20230105_ID4301_C_04_N00113.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221114_ID1691_C_04_N00205.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221202_ID2830_C_03_N00116.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221121_ID2114_C_04_N00207.png가 train 폴더로 JPG로 변환되어 복사되었습니다.
이미지 20221219_ID3648_C_01_N00106.png가 tra

In [ ]:
# PNG 파일을 삭제할 디렉토리 경로 설정
directories = [
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img',
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img',
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img'
]

# 각 디렉토리에서 PNG 파일 삭제
for directory in directories:
    for filename in os.listdir(directory):
        if filename.endswith(".png"):  # 확장자가 PNG인 파일 찾기
            file_path = os.path.join(directory, filename)
            os.remove(file_path)  # 파일 삭제
            print(f"{file_path} 파일이 삭제되었습니다.")

print("모든 PNG 파일이 삭제되었습니다.")


/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20221214_ID3598_B_01_N00030.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20220824_ID0008_B_04_N00032.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20221129_ID2339_B_01_N00270.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20221212_ID3401_B_04_N00054.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20221018_ID0257_B_02_N00193.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20230107_ID4121_B_04_N00027.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img/20220921_ID0116_B_04_N00129.png 파일이 삭제되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_c

In [ ]:
### 라벨 병합

In [7]:
# 병합할 JSON 파일 경로 설정
merge_pairs = [
    {
        "aihub": '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label/combined_test_annotations.json',
        "est": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v1.json',
        "output": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json'
    },
    {
        "aihub": '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label/combined_train_annotations.json',
        "est": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v1.json',
        "output": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json'
    },
    {
        "aihub": '/content/drive/MyDrive/project/KDT_kiosk/data/aihub_label/combined_val_annotations.json',
        "est": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v1.json',
        "output": '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json'
    }
]

In [3]:
# 각 파일 쌍에 대해 병합 수행
for pair in merge_pairs:
    # 각 파일 읽기
    with open(pair["aihub"], 'r') as file:
        aihub_data = json.load(file)

    with open(pair["est"], 'r') as file:
        est_data = json.load(file)

    # 두 리스트 병합
    merged_data = aihub_data + est_data

    # 병합된 데이터를 새로운 파일에 저장
    with open(pair["output"], 'w') as file:
        json.dump(merged_data, file, indent=4)

    print(f"{pair['output']} 파일이 병합되어 생성되었습니다.")


/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json 파일이 병합되어 생성되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json 파일이 병합되어 생성되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json 파일이 병합되어 생성되었습니다.


In [ ]:
'/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json'
'/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json'
'/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json'

In [10]:
# 파일 경로 목록
json_paths = [
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json',
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json',
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json'
]

# 각 파일에서 image 필드의 확장자를 .png에서 .jpg로 변경
def update_image_extension(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)

    updated = False  # 변경 여부를 확인하기 위한 플래그
    for item in data:
        if item['image'].endswith('.png'):
            item['image'] = item['image'].replace('.png', '.jpg')
            updated = True  # 변경된 경우 플래그 설정

    # 변경된 경우에만 파일을 다시 저장
    if updated:
        with open(json_path, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"{json_path} 파일의 .png 확장자가 .jpg로 변경되었습니다.")
    else:
        print(f"{json_path} 파일에 변경할 .png 확장자가 없습니다.")

# 각 JSON 파일에 대해 처리
for json_path in json_paths:
    update_image_extension(json_path)

print("이미지 확장자가 .png에서 .jpg로 변경되었습니다.")


/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json 파일의 .png 확장자가 .jpg로 변경되었습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json 파일에 변경할 .png 확장자가 없습니다.
/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json 파일에 변경할 .png 확장자가 없습니다.
이미지 확장자가 .png에서 .jpg로 변경되었습니다.


In [4]:
# 잘 이동했는지 확인

final_est_test_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img'
final_est_train_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img'
final_est_val_img_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img'

# 파일 개수 확인
final_utk_test_img_cnt = len([name for name in os.listdir(final_est_test_img_path) if os.path.isfile(os.path.join(final_est_test_img_path, name))])
final_utk_train_img_cnt = len([name for name in os.listdir(final_est_train_img_path) if os.path.isfile(os.path.join(final_est_train_img_path, name))])
final_utk_val_img_cnt = len([name for name in os.listdir(final_est_val_img_path) if os.path.isfile(os.path.join(final_est_val_img_path, name))])
print(f"파일 개수: {final_utk_test_img_cnt} {final_utk_train_img_cnt} {final_utk_val_img_cnt}")

파일 개수: 2100 10494 2100


In [5]:
train_label_path ='/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json'
test_label_path ='/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json'
val_label_path ='/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json'
with open(train_label_path, 'r') as file:
    train_data = json.load(file)
with open(test_label_path, 'r') as file:
    test_data = json.load(file)
with open(val_label_path, 'r') as file:
    val_data = json.load(file)
len(train_data), len(test_data), len(val_data)

(2100, 10494, 2100)

In [6]:
train_data

[{'image': '20221214_ID3598_B_01_N00030.png',
  'gender': 'F',
  'age': 2,
  'emotion': 'surprise',
  'race': 'oriental',
  'box': {'x': 298, 'y': 303, 'w': 381, 'h': 509}},
 {'image': '20220824_ID0008_B_04_N00032.png',
  'gender': 'F',
  'age': 1,
  'emotion': 'surprise',
  'race': 'oriental',
  'box': {'x': 335, 'y': 281, 'w': 399, 'h': 551}},
 {'image': '20221129_ID2339_B_01_N00270.png',
  'gender': 'M',
  'age': 1,
  'emotion': 'surprise',
  'race': 'oriental',
  'box': {'x': 408, 'y': 287, 'w': 363, 'h': 548}},
 {'image': '20221212_ID3401_B_04_N00054.png',
  'gender': 'M',
  'age': 1,
  'emotion': 'surprise',
  'race': 'oriental',
  'box': {'x': 260, 'y': 269, 'w': 401, 'h': 542}},
 {'image': '20221018_ID0257_B_02_N00193.png',
  'gender': 'F',
  'age': 1,
  'emotion': 'surprise',
  'race': 'oriental',
  'box': {'x': 239, 'y': 251, 'w': 529, 'h': 616}},
 {'image': '20230107_ID4121_B_04_N00027.png',
  'gender': 'M',
  'age': 1,
  'emotion': 'surprise',
  'race': 'oriental',
  'box':

In [15]:
json_file_pathes = ['/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json',
                    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json',
                    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json',
                    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_non_emo_test_label/final_test_labels.json',
                    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_non_emo_train_label/final_train_labels.json',
                    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_non_emo_val_label/final_val_labels.json'
                    ]

In [16]:
# emotion 매핑 정의
emotion_mapping = {
    'surprise': 'surprise',
    'happy': 'happy',
    'sadness': 'sad',
    'fear': 'fear',
    'anger': 'angry',
    'disgust': 'disgust',
    'none': 'neutral'
}

for json_file_path in json_file_pathes:
  # JSON 파일 로드
  with open(json_file_path, 'r') as f:
      annotations = json.load(f)

  # emotion 필드를 새로운 매핑에 맞게 변경 (emotion 필드가 있는지 먼저 확인)
  for item in annotations:
      if 'emotion' in item:  # emotion 필드가 있는지 확인
          if item['emotion'] in emotion_mapping:
              item['emotion'] = emotion_mapping[item['emotion']]

  # 변경된 내용을 동일한 파일에 다시 저장
  with open(json_file_path, 'w') as f:
      json.dump(annotations, f, indent=4)

  print("Emotion field updated successfully.")


Emotion field updated successfully.
Emotion field updated successfully.
Emotion field updated successfully.
Emotion field updated successfully.
Emotion field updated successfully.
Emotion field updated successfully.


In [14]:
# 어노테이션 image에서 전체 경로 제거하고 파일명만 남김

for json_file_path in json_file_pathes:
  # JSON 파일 로드
  with open(json_file_path, 'r') as f:
      annotations = json.load(f)

  # image 경로에서 파일명만 남기도록 수정
  for item in annotations:
      # 이미지 경로에서 파일명만 남기기
      if '/' in item['image']:
          item['image'] = os.path.basename(item['image'])

  # 변경된 내용을 동일한 파일에 다시 저장
  with open(json_file_path, 'w') as f:
      json.dump(annotations, f, indent=4)

  print("Image paths updated successfully.")


Image paths updated successfully.
Image paths updated successfully.
Image paths updated successfully.
Image paths updated successfully.
Image paths updated successfully.
Image paths updated successfully.


#### 이후 all_code.ipynb 참조

# 모델 훈련

In [6]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
from networks.DDAM import DDAMNet
from tqdm import tqdm
import json
import os
from torch.utils.data import Dataset
import wandb

In [7]:
# wandb 접속
wandb.login(key='ec9a405befa33740a45134d63bb3ba44febe52d4')

wandb.init(project="est_final_project")
# PyTorch 및 관련 패키지 임포트


# 디바이스 설정 (GPU 사용 가능 시)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: happybok (happybok-korea). Use `wandb login --relogin` to force relogin


In [8]:
class EmotionDataset(Dataset):
    def __init__(self, json_file, image_dir, transform=None):
        # JSON 파일 로드
        with open(json_file, 'r') as f:
            self.annotations = json.load(f)

        self.image_dir = image_dir
        self.transform = transform

        # 감정 라벨 맵핑
        self.emotion_mapping = {
            "neutral": 0,
            "happy": 1,
            "sad": 2,
            "surprise": 3,
            "fear": 4,
            "disgust": 5,
            "angry": 6
        }

        # 'none' 감정 값 필터링
        self.annotations = [item for item in self.annotations if item["emotion"] != 'none']

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # 이미지 파일명과 경로
        img_info = self.annotations[idx]
        img_name = img_info["image"]
        img_path = os.path.join(self.image_dir, img_name)

        # 이미지 로드
        image = Image.open(img_path).convert('RGB')

        # 감정 레이블 추출
        emotion = img_info["emotion"]
        label = self.emotion_mapping.get(emotion, -1)

        # 감정 매핑에 없는 경우 처리
        if label == -1:
            raise ValueError(f"Unknown emotion: {emotion}")

        if self.transform:
            image = self.transform(image)

        return image, label


In [9]:
# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # 모델 입력 크기에 맞게 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# 경로 설정
train_image_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_img'
val_image_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_img'
train_json_file = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_train_label/final_labels_v2.json'
val_json_file = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_val_label/final_val_labels_v2.json'

# 데이터셋 및 데이터로더 생성
train_dataset = EmotionDataset(train_json_file, train_image_dir, transform=transform)
val_dataset = EmotionDataset(val_json_file, val_image_dir, transform=transform)

# 데이터로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [10]:
model = DDAMNet(num_class=7, num_head=2, pretrained=True).to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 학습률 스케줄러 설정
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

/content/drive/MyDrive/project/KDT_kiosk/code/extracode/DDAMFN/DDAMFN++/networks/DDAM.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net = torch.load(os.path.join('./p

In [11]:
# 학습 함수 정의
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # 모델을 학습 모드로 전환
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(tqdm(train_loader)):
            inputs, labels = inputs.to(device), labels.to(device)

            # 옵티마이저 초기화
            optimizer.zero_grad()

            # 순전파
            outputs, _, _ = model(inputs)
            loss = criterion(outputs, labels)

            # 역전파 및 최적화
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

             # 학습 예측값과 정확도 계산
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # 학습 정확도 계산
            train_accuracy = 100 * correct / total

            # wandb에 학습 손실 및 학습률 로깅
            wandb.log({
                "Train Loss": loss.item(),
                "Learning Rate": optimizer.param_groups[0]['lr'],
                "Train Accuracy": train_accuracy
                })

            # 학습 중간에 이미지와 예측값 로깅
            if i % 100 == 0:  # 매 100번째 배치마다
                wandb.log({"Input Images": [wandb.Image(inputs[j], caption=f"Label: {labels[j]}, Prediction: {torch.argmax(outputs[j])}") for j in range(min(4, inputs.size(0)))]})
        # 학습 에폭 당 정확도 계산
        train_accuracy = 100 * correct / total

        # 학습률 스케줄러 업데이트
        scheduler.step()

        # 에폭 당 손실 및 정확도 출력
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Train Accuracy: {train_accuracy:.2f}%')


        # wandb에 학습 정확도 및 손실 로깅
        wandb.log({"Train Accuracy": train_accuracy, "Train Loss": running_loss / len(train_loader)})

        # 검증 성능 평가
        validate_model(model, val_loader, criterion)

# 검증 함수 정의
def validate_model(model, val_loader, criterion):
    model.eval()  # 평가 모드로 전환
    correct = 0
    total = 0
    val_loss = 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs, _, _ = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # 예측값과 정확도 계산
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

    # wandb에 검증 성능 및 이미지와 예측 결과 로깅
    wandb.log({"Validation Accuracy": accuracy, "Validation Loss": val_loss / len(val_loader)})

    # wandb에 Confusion Matrix 로깅
    wandb.log({"Confusion Matrix": wandb.plot.confusion_matrix(probs=None,
                                                              y_true=all_labels,
                                                              preds=all_preds,
                                                              class_names=["Neutral", "Happy", "Sad", "Surprise", "Fear", "Disgust", "Angry"])})


In [ ]:
# 모델 학습 실행
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

 65%|██████▍   | 212/328 [35:55<18:29,  9.57s/it]

In [ ]:
# 학습 후 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/project/KDT_kiosk/model/ddamfn/fine_tuned_ddamfn.pth')

# wandb에 모델 저장
wandb.save('/content/drive/MyDrive/project/KDT_kiosk/model/ddamfn/fine_tuned_ddamfn.pth')

wandb.finish()

In [ ]:
# 런타임 종료

import os

# 모델 학습 및 작업이 모두 완료된 후 실행
print("작업이 완료되었습니다. 런타임을 해제합니다.")

# 코랩 런타임 해제 명령어
os._exit(0)


# 파인튜닝한것으로 추론

In [ ]:
import torch
from networks.DDAM import DDAMNet  # 모델 클래스 임포트

# 감정 매핑 (모델의 출력에 따라 감정 라벨을 매핑)
emotion_labels = ['neutral', 'happy', 'sad', 'surprise', 'fear', 'disgust', 'angry']

# 디바이스 설정 (GPU 사용 가능 시)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 초기화
model = DDAMNet(num_class=7, num_head=2)

# 저장된 모델 가중치 로드
model.load_state_dict(torch.load('/content/drive/MyDrive/project/KDT_kiosk/model/ddamfn/fine_tuned_ddamfn.pth'))
model.to(device)

# 모델을 평가 모드로 전환
model.eval()


In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import json

# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# 테스트용 데이터셋 정의
class TestEmotionDataset(Dataset):
    def __init__(self, json_file, image_dir, transform=None):
        with open(json_file, 'r') as f:
            self.annotations = json.load(f)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_info = self.annotations[idx]
        img_name = img_info['image']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_info['emotion']  # 라벨은 비교를 위해 반환

# 경로 설정
test_image_dir = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_img'
test_json_file = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/final_test_label/final_test_labels_v2.json'

# 데이터셋 및 데이터로더 생성
test_dataset = TestEmotionDataset(test_json_file, test_image_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# 추론 함수 정의
def predict(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, true_emotions in test_loader:
            images = images.to(device)
            outputs, _, _ = model(images)
            _, predicted = torch.max(outputs, 1)

            # 예측한 감정을 감정 라벨로 변환
            predicted_emotions = [emotion_labels[p.item()] for p in predicted]
            predictions.extend(predicted_emotions)

    return predictions

# 추론 실행
predictions = predict(model, test_loader)

# 결과 출력
for idx, (image_info, predicted_emotion) in enumerate(zip(test_dataset.annotations, predictions)):
    print(f"Image: {image_info['image']}, Predicted Emotion: {predicted_emotion}")
